In [ ]:
import os, sys, getopt 
os.environ['KERAS_BACKEND'] = 'theano'
import yaml
import cPickle
import numpy as np
import keras
from matplotlib import pyplot as plt 
import keras.callbacks
from keras.datasets import mnist
#from keras.utils.visualize_util import plot
from keras.models import Sequential, Model
from keras.optimizers import SGD, Adam, RMSprop
from model import build_model, MyCallback
from keras.callbacks import CSVLogger
from data import load
from sklearn.model_selection import train_test_split
import keras.backend as K
import theano.tensor as T
from theano import function
#np.set_printoptions(threshold=10000000)
from theano.printing import return_value,reset_value
keras.__version__

In [ ]:
%run model.ipynb

In [ ]:
dataset = '803262'

In [ ]:

def build_data_dict(in_name, out_name, in_data, out_data):
    in_dict = dict()
    in_dict[in_name] = in_data
    
    out_dict = dict((k, out_data) for k in out_name)
    return (in_dict, out_dict)

def build_loss_weight(config):
    if config['hedge'] == False:
        w = [1.]
    elif config['loss_weight'] == 'ave':
        w = [1./ config['n_layers']]* config['n_layers']
    return w
def eer(y_true, y_pred):
    y_true.eval()
    #print type(K.mean(y_pred))
    x1 = T.dscalar('x')
    y1 = T.dscalar('x')
    f = function([x], z)
    
    return K.mean(y_pred)

def binary_accuracy(y_true, y_pred):
    
    '''Calculates the mean accuracy rate across all predictions for binary
    classification problems.
    '''
    if print_mode==1:
        y_pred = K.print_tensor(y_pred)
    #K.print_tensor(y_pred)
    #K.get_value(y_pred)
    return K.mean(K.equal(y_true, K.round(y_pred)))

In [ ]:
def main(arg, idx=0):
    config = {'learning_rate': 1e-3,
              'optim': 'Adam',
              'batch_size': 1,
              'nb_epoch': 50,
              'n_layers': 10,
              'hidden_num': 100,
              'activation': 'relu',
              'loss_weight': 'ave',
              'adaptive_weight': False,
              'data': 'mnist',
              'hedge': True,
              'log': 'mnist_hedge.log'}

    configfile = ''
    opts = [('-c', 'hb19.yaml')]
    args = []
    for opt, arg in opts:
        if opt == '-h':
            print (helpstring)
            yamlstring = yaml.dump(config,default_flow_style=False,explicit_start=True)
            print("YAML configuration file format:")
            print("")
            print("%YAML 1.2")
            print(yamlstring)
            sys.exit()

        elif opt in ('-c', '--config'):
            configfile = arg

        print("Config file is %s" % configfile)

    if os.path.exists(configfile):
        f = open(configfile)
        user_config = yaml.load(f.read())
        config.update(user_config)
    
    print("Printing configuration:")
    for key,value in config.iteritems():
        print("  ",key,": ",value)
        
    (X_train, Y_train, X_test, Y_test, nb_classes) = load(dataset)
    (_, _, x_test, y_test, nb_classes) = load(dataset)
    
    print np.array_equal(Y_test,y_test)
    
    Y_TRAIN = Y_train.reshape(-1)
    Y_TEST = Y_test.reshape(-1)
    
    train_len = Y_TRAIN.shape[0]
    test_len = Y_TEST.shape[0]
    
    Y_train = np.zeros((train_len,2))
    Y_test = np.zeros((test_len,2))
    
    Y_train[np.arange(train_len), Y_TRAIN] = 1
    Y_test[np.arange(test_len), Y_TEST] = 1
    
    
    #(X_test, Y_test, X_train, Y_train, nb_classes) = load(config['data'])
    #print Y_test
    #X_train = X_train[:500000]
    #Y_train = Y_train[:500000]
    '''
    X_test = X_test[:1]
    Y_test = Y_test[:1]'''
    '''
    # data for concept drift data 
    import numpy as np

    X = np.load('/home/rakesh/Desktop/zCap/x_train.npy')
    y = np.load('/home/rakesh/Desktop/zCap/y_train.npy')
    
    X_test, X_train, Y_test, Y_train = train_test_split(X, y,
                                                    stratify=y, 
                                                    test_size=0.211)
    '''
    model, in_name, out_name = build_model(config)
    if len(out_name) == 1:
        out_name_loss = ['loss']
    else:
        out_name_loss = [s + '_loss' for s in out_name]

    model.summary()
    
    #print Y_test[:,1].sum()
    
    #plot(model, to_file = 'model.png')
    
    optim = eval(config['optim'])(lr = config['learning_rate'])
    in_dict, out_dict = build_data_dict(in_name, out_name, X_train, Y_train)
    in_val, out_val = build_data_dict(in_name, out_name, X_test, Y_test)
    loss_dict = dict((k, 'categorical_crossentropy') for k in out_name) 
  
    loss_weights = build_loss_weight(config)
    my_callback = MyCallback( loss_weights, names = out_name_loss, hedge = config['hedge'], log_name = config['log'])
    #csv  = CSVLogger(config['log'])
    model.compile(optimizer = optim, loss = loss_dict, hedge = config['hedge'],loss_weights = loss_weights, metrics =['accuracy'])#,binary_accuracy]) #['accuracy'])
    history = model.fit(in_dict, out_dict,validation_split=0.2, nb_epoch = config['nb_epoch'], batch_size = config['batch_size'], callbacks=[my_callback] , verbose = 1)
    #history = model.fit(in_val, out_val, nb_epoch = 1, batch_size = 10, callbacks=[my_callback] , verbose = 1)
    #print(len(k))
    #print model.evaluate(X_test,Y_test)
    #print my_callback.acc
    
    k=return_value()
    #print(k[0].shape)
    cumLoss = np.cumsum(my_callback.acc)
    indexOfLoss = np.arange(len(cumLoss))+1
    cumAverageLoss = cumLoss/indexOfLoss
    filename = (config['log'] + '_' + str(idx) + '.acc')
    np.savetxt(filename, cumAverageLoss, delimiter=',')
    
    return my_callback,history,model,x_test,y_test,in_val,out_val,k

In [ ]:
history = None
Model1 = None
X_test = None
Y_test = None
in_val = None
out_val = None
my_callback = None
k=None
if __name__ == '__main__':
    #for i in range(5):
    my_callback,history,Model1,X_test,Y_test,in_val,out_val,k = main(sys.argv[1:], 0)

In [ ]:
reset_value()                    ## clears prediction buffer
print return_value()                            
metrics = Model1.evaluate(in_val,out_val,verbose=1)
k=return_value()                                  # predictions from model

In [ ]:
## test metrics
print("Test Accuracy : ",metrics[-1])
metrics

In [ ]:
y_pred = np.concatenate(k)
y_true = out_val[out_val.keys()[-1]]

## ROC

In [ ]:
from sklearn.metrics import roc_auc_score,roc_curve,auc

In [ ]:
y_scores = y_pred[:,1]
y_true = y_true[:,1]
y_true.shape

In [ ]:
fpr, tpr, thresholds = roc_curve(y_true, y_scores, pos_label=1)
fnr = 1 - tpr
EER = fpr[np.nanargmin(np.absolute((fnr - fpr)))]

In [ ]:
print "FPR",fpr.shape
print "TPR",tpr.shape
print "thresholds",thresholds

In [ ]:
print "AUC",roc_auc_score(y_true, y_scores)
print "EER",EER

In [ ]:
fnr[np.nanargmin(np.absolute((fnr - fpr)))]

## ROC Curve

In [ ]:
from scipy.optimize import brentq
from scipy.interpolate import interp1d
eer = brentq(lambda x : 1. - x - interp1d(fpr, tpr)(x), 0., 1.)
thresh = interp1d(fpr, thresholds)(eer)
print "EER",eer

In [ ]:
figure = plt.gcf() # get current figure
figure.set_size_inches(8, 6)
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc_score(y_true, y_scores))

plt.plot(fpr, tpr, 'b', label = 'EER = %0.2f' % eer)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([-0.02, 1.02])
plt.ylim([-0.02, 1.02])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title(dataset+' ROC - 50ms ')
plt.legend(loc="lower right")
plt.savefig(dataset+'_plot_hbp_roc.png',dpi=100)

plt.clf() 

In [ ]:
## Error rate whil whole training

hbp = open('/home/rakesh/Desktop/zCap/new/log_hbp19_0.acc','r')

xhbp = []

for line in hbp.readlines():
    xhbp.append(float(line))
    
    
figure = plt.gcf() # get current figure
figure.set_size_inches(8, 6)

plt.plot(1-np.array(xhbp), label = 'HBP')
plt.legend()
plt.title(dataset+' 50ms 21 features HBP')
plt.ylabel('Error Rate')
#plt.show()
plt.savefig(dataset+'_plot_hbp_cd.png',dpi=100)

plt.clf()  

In [ ]:
##v  train and val curve
figure = plt.gcf() # get current figure
figure.set_size_inches(8, 6)

plt.plot(np.array(history.history['weighted_acc']), label = 'Training')
plt.plot(np.array(history.history['val_weighted_acc']), label = 'Validation')
plt.legend()
plt.title(dataset+' | 50ms | Test_acc : %0.3f'%metrics[-1])
plt.ylabel('Accuracy')
#plt.show()
plt.savefig(dataset+'_plot_train_val.png',dpi=100)

plt.clf()